# Full Search

## Load Bible Text

In [1]:
import numpy as np
import pandas as pd

types = {
    'book': 'string',
    'chap': np.int32,
    'vers': np.int32,
    'text': 'string'
}
unv = pd.read_csv(
    'https://bible.fhl.net/public/dnstrunv.tgz',
    sep='#',
    compression='gzip',
    header=None,
    usecols=[1, 2, 3, 4],
    names=list(types.keys()))

In [58]:
unv.dropna(inplace=True)
unv = unv.astype(types)
unv.sort_values(['book', 'chap', 'vers'], inplace=True)

def in_order(nums: np.ndarray):
    return np.array_equal(nums, range(1, nums.max() + 1))

assert len(unv.book.unique()) == 66
assert unv.groupby('book').chap.unique().apply(in_order).all()
assert unv.groupby(['book', 'chap']).vers.apply(in_order).all()
assert unv.groupby(['book', 'chap']).vers.max().sum() == len(unv) == 31103

## Tokenize

In [5]:
import jieba
import jieba.posseg as pseg

jieba.enable_paddle()

Paddle enabled successfully......


In [6]:
from hanziconv import HanziConv

unv['s_text'] = unv.text.apply(HanziConv.toSimplified)

In [7]:
unv['s_text_tk'] = unv.s_text.apply(lambda v: pseg.lcut(v, use_paddle=True))

## Search

In [9]:
from typing import Iterable

def highlight_occurances(text: str, keywords: Iterable[str]) -> str:
    for i, kw in enumerate(keywords):
        text = text.replace(kw, highlight(kw, i))
    return text

def highlight(text: str, color_code: int) -> str:
    return f'\x1b[6;30;4{color_code + 1}m{text}\x1b[0m'

test_data = ['first', 'second', 'thrid', 'forth', 'fifth', 'sixth', 'seventh']
print(highlight_occurances(', '.join(test_data), test_data))

first, second, thrid, forth, fifth, sixth, seventh


In [10]:
from search import sentence_similarity

searches = ['挂虑 祈祷', '喜乐 事奉', '求救', '信心 行事']

for search_term in searches:
    print(f'Search for {search_term}:')
    search_tk = pseg.lcut(search_term, use_paddle=True)
    match_scores = {}
    for v, vers_tk in zip(unv.s_text, unv.s_text_tk):
        similarity = sentence_similarity(search_tk, vers_tk)
        match_kw = [kw for kw in similarity.keys() if similarity[kw] > 0]
        vers = highlight_occurances(v, match_kw)
        score = sum(similarity.values())
        match_scores[vers] = score
    for top_match in sorted(match_scores, key=match_scores.get, reverse=True)[:10]:
        print(f'Match: {match_scores[top_match]:7.4f} Verse: {top_match}')
    print()

Search for 挂虑 祈祷:
Match:  2.0000 Verse: 应当一无挂虑，只要凡事借着祷告、祈求，和感谢，将你们所要的告诉神。
Match:  1.1429 Verse: 妇人和处女也有分别。没有出嫁的，是为主的事挂虑，要身体、灵魂都圣洁；已经出嫁的，是为世上的事挂虑，想怎样叫丈夫喜悦。
Match:  1.1429 Verse: 我愿你们无所挂虑。没有娶妻的，是为主的事挂虑，想怎样叫主喜悦。
Match:  1.1429 Verse: 娶了妻的，是为世上的事挂虑，想怎样叫妻子喜悦。
Match:  1.1429 Verse: 他必像树栽于水旁，在河边扎根，炎热来到，并不惧怕，叶子仍必青翠，在干旱之年毫无挂虑，而且结果不止。
Match:  1.0000 Verse: 在这日以前，这日以后，耶和华听人的祷告，没有像这日的，是因耶和华为以色列争战。
Match:  1.0000 Verse: 于是禁食祷告，按手在他们头上，就打发他们去了。
Match:  1.0000 Verse: 希西家就转脸朝墙，祷告耶和华说：
Match:  1.0000 Verse: 散了众人以后，他就独自上山去祷告。到了晚上，只有他一人在那里。
Match:  1.0000 Verse: 对他们说：「经上说：我的殿必作祷告的殿，你们倒使它成为贼窝了。」

Search for 喜乐 事奉:
Match:  2.0000 Verse: 你们当乐意事奉耶和华，当来向他歌唱！
Match:  2.0000 Verse: 「以色列家啊，至于你们，主耶和华如此说：从此以后若不听从我，就任凭你们去事奉偶像，只是不可再因你们的供物和偶像亵渎我的圣名。
Match:  2.0000 Verse: 「我儿所罗门哪，你当认识耶和华─你父的神，诚心乐意地事奉他；因为他鉴察众人的心，知道一切心思意念。你若寻求他，他必使你寻见；你若离弃他，他必永远丢弃你。
Match:  2.0000 Verse: 那时，我必使万民用清洁的言语好求告我─耶和华的名，同心合意地事奉我。
Match:  2.0000 Verse: 暗兰的儿子是亚伦、摩西。亚伦和他的子孙分出来，好分别至圣的物，在耶和华面前烧香、事奉他，奉他的名祝福，直到永远。
Match:  2.0000 Verse: 就能废弃我与我仆人大卫所立的约